In [ ]:
# Importing essential libraries
from flask import Flask, render_template, request, redirect, url_for, flash, session, redirect
import logging, re, os
from datetime import datetime
from werkzeug.security import generate_password_hash, check_password_hash 
import sqlite3

# ------------------------------------
#Initialize Flask Application
app = Flask(__name__)
# ------------------------------------


# ------------------------------------
# In production set SECRET_KEY via environment variable
app.secret_key = os.environ.get("SECRET_KEY", "dev-change-me")
# ------------------------------------

# ------------------------------------
# Database path
DB_PATH = "hospitaldata.db"
# ------------------------------------

# ------------------------------------
#Creating user table if they don't exist
def init_db():
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()
    cursor.execute("""
    CREATE TABLE IF NOT EXISTS user (
        ID INTEGER PRIMARY KEY AUTOINCREMENT,
        First_name TEXT NOT NULL,
        Last_name TEXT NOT NULL,
        username TEXT NOT NULL,
        email TEXT NOT NULL,
        password TEXT NOT NULL,
        role TEXT NOT NULL,
        created_at TIMESTAMP DEFAULT CURRENT_TIMESTAMP
    );
    """)

    #Case-sensitive uniqueness on email
    cursor.execute("""
    CREATE UNIQUE INDEX IF NOT EXISTS idx_users_email_nocase
    ON user (lower(email));
    """)

    #Creating 5 admins
    admins = [
        {'first':'Charlie', 'last':'Alpha', 'user': 'admin1', 'email':'admin1@hospital.com', 'pass':'Admin123!'},
        {'first':'Max', 'last':'Beta', 'user': 'admin2', 'email':'admin2@hospital.com', 'pass':'Admin123?'},
        {'first':'Jade', 'last':'Gamma', 'user': 'admin3', 'email':'admin3@hospital.com', 'pass':'Admin456!'},
        {'first':'Amy', 'last':'Delta', 'user': 'admin4', 'email':'admin4@hospital.com', 'pass':'Admin456?'},
        {'first':'Bob', 'last':'Epsilon', 'user': 'admin5', 'email':'admin5@hospital.com', 'pass':'Admin789!'},
    ]

    #Loop to iterate through the list of admins for database insertion
    for admin in admins:
        cursor.execute("SELECT 1 FROM user WHERE username = ?", (admin['user'],))

        if cursor.fetchone() is None:

            #hash the passwords for security 
            hashed_pw = generate_password_hash(admin['pass'], method="pbkdf2:sha256")
            insert_query = """
            INSERT INTO user (First_name, Last_name, username, email, password, role)
            VALUES (?, ?, ?, ?, ?, ?)
            """

            #Preparing the tuple for insertion:
            user_values = (
                admin['first'],
                admin['last'],
                admin['user'],
                admin['email'],
                hashed_pw,
                'admin' #hardcoding the role as admin
            )

            cursor.execute(insert_query, user_values)
            print(f"Created user: {admin['user']}")
        else:
            print(f"User already exists: {admin['user']}")

    conn.commit()
    conn.close()

# ------------------------------------
#Configure Logging
logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s [%(levelname)s] %(message)s",
    handlers=[logging.FileHandler("app.log"), logging.StreamHandler()],
)
log = logging.getLogger(__name__)
# ------------------------------------


# ------------------------------------
# Simple in-memory storage
REGISTERED_USERS = [] # each item: {"username", "email", "age", "created_at"}
# ------------------------------------


# ------------------------------------
# Validation patterns

##Ensure that first name is only letters and hyphens
FIRSTNAME_PATTERN = re.compile(r'^[A-Za-z-]{1,50}$')

##Ensure that last name is only letters and hyphens
LASTNAME_PATTERN = re.compile(r'[A-Za-z-]{1,50}$')

## Ensures the email has a basic valid structure of name@domain.tld
EMAIL_PATTERN = re.compile(r'^[\w\.-]+@[\w\.-]+\.[A-Za-z]{2,}$')

## Ensures the username only allows letters, digits, underscores, and length between 5 and 16 characters.
USERNAME_PATTERN = re.compile(r'^[A-Za-z0-9_]{5,16}$')

## Strong password pattern that requires lowercase, uppercase, digit, special character, and minimum 8 characters.
PASSWORD_PATTERN = re.compile(r'^(?=.*[a-z])(?=.*[A-Z])(?=.*\d)(?=.*[@$!%*?&]).{8,}$')
# ------------------------------------


# ------------------------------------
# Route for the Home page
@app.route('/')
def home():
    return render_template('Home_Page.html')

# Route for the About page
@app.route('/about')
def about():
    return render_template('About.html')

# Route for Registration success page 
@app.route('/success')
def success():
    return render_template('Registration_Success.html')

# Route for User Registration page
## Simple register form that intakes first name, last name, username, email and password for now 
@app.route('/register', methods=['GET', 'POST'])
def register():
    if request.method == 'POST':
        First_name = (request.form.get("First_name") or "").strip()
        Last_name = (request.form.get("Last_name") or "").strip()
        username = (request.form.get("username") or "").strip()
        email    = (request.form.get("email") or "").strip()
        password = (request.form.get("password") or "").strip()
        
        try:

            ## ---- SERVER-SIDE INPUT VALIDATION FOR REGISTRATION FORM ---- ##

            # ----- Checking empty fields -----
             if not First_name:
                 raise ValueError("First name is required")
             if not Last_name:
                 raise ValueError("Last name is required")
             if not username:
                 raise ValueError("Username is required")
             if not email:
                 raise ValueError("Email is required.")
             if not password:
                 raise ValueError("Password is required.")
             
            # ----- Type / Format checks -----
             if not FIRSTNAME_PATTERN.fullmatch(First_name):
                  raise ValueError("First name must only contain letters and hyphens")

             if not LASTNAME_PATTERN.fullmatch(Last_name):
                  raise ValueError("Last name must only contain letters and hyphens")
             
             if not USERNAME_PATTERN.fullmatch(username):
                  raise ValueError("Username must be 5–16 chars (letters, digits, underscore).")
             
             if not EMAIL_PATTERN.fullmatch(email):
                  raise ValueError("Email format is invalid.")
             
             if not PASSWORD_PATTERN.fullmatch(password):
                  raise ValueError("Password format is invalid.")
             
             if len(email) > 254:
                  raise ValueError("Email too long.")

            # ----- Business rules / Whitelist checks -----
             if username.lower() in {"admin", "root"}:
                 raise ValueError("Username is reserved.")

            # ----- Hashing password ------
             hashed_password = generate_password_hash(password, method="pbkdf2:sha256")

             conn= sqlite3.connect(DB_PATH)
             cursor = conn.cursor()

            # ----- Pre-check duplicate -----
             cursor.execute("SELECT 1 FROM doctor WHERE lower(email) = lower(?)", (email,))
             if cursor.fetchone():
                 flash("This email is already registered. Please log in instead.")
                 conn.close()
                 return redirect(url_for("login"))
             
            # ----- Insert the user into the database ----- 
             try:
                 cursor.execute("""
                    INSERT INTO user (First_name, Last_name, username, email, password, role)
                    VALUES (?, ?, ?, ?, ?, ?)
                 """, (First_name, Last_name, username, email, hashed_password, 'doctor')) #doctor is set as default role for role based access 
                 conn.commit()
                 flash("Registration successful! Please log in.")
             except sqlite3.IntegrityError: 
                 flash("This email is already registered. Please log in.")
             finally:
                 conn.close()

             return redirect(url_for("success"))
        
        except ValueError as e:
            flash(str(e), 'error')
            log.warning("Validation failed: %s", e)
            return redirect(url_for("register"))
       
    return render_template("Registration_Form.html")

# Route for Login page and Submissing Handling
@app.route('/login', methods=['GET', 'POST'])
def login():
    if request.method == "GET":
        return render_template("Login.html")
    
    # ----- Post request handling -----
    username = request.form.get("username", "").strip()
    password = request.form.get("password", "")

    if not (username and password):
        flash("Please enter username and password.")
        return redirect(url_for("login"))
    
    # ----- Database connection and Query -----
    conn = sqlite3.connect(DB_PATH)
    cursor = conn.cursor()

    #Querying the doctor table to find a user matching the username 
    cursor.execute("""
        SELECT ID, First_name, Last_name, username, email, password
        FROM user
        WHERE lower(username) = lower(?)
    """, (username,))
    row = cursor.fetchone()
    conn.close() #Close connection immediately after fetching data

    # ----- Check if we found a user in the database
    if not row:
        flash("Invalid username or password.")
        return redirect(url_for("login"))
    
    # ----- Map retrieved columns to variables -----
    ID, First_name, Last_name, username, email, password_hash, user_role = row

    # ----- Verify the submitted password against the stored hash -----
    if not check_password_hash(password_hash, password):
        flash("Invalid email or password.")
        return redirect(url_for("login"))
    
    # ----- Upon successful login, create Session -----
    session["ID"] = ID
    session["email"] = email
    session["user_name"] = f"{First_name} {Last_name}"
    session["role"] = user_role
    flash(f"Welcome back!")
    return redirect(url_for('dashboard'))

#Dashboard page for an authenticated user who has logged in
#Role Based Access Control (RBAC) implemented
#Doctors and Admins are redirected to different dashboards due to their roles
@app.route("/dashboard")
def dashboard():
    if "ID" not in session:
        flash("Please log in to continue.")
        return redirect(url_for("login"))
    
    #retrieve role
    user_role = session.get('role')
    
    if user_role == 'admin': 
        #Admin views the admin dashboard page
        return redirect(url_for('admin_dashboard'))
    elif user_role == 'doctor':
        #Doctor views the doctor dashboard
        return render_template("dashboard.html")
    else: 
        #Default response for unrecognized roles
        flash("Your user role is unrecognized. Please contact support.", 'error')
        return redirect(url_for("login"))

#Admin dashboard
@app.route('/admin_dashboard')
def admin_dashboard():
    return render_template('admin_dashboard.html')

#Re routing user to login page when user logs out
@app.route("/logout")
def logout():
    session.clear()
    flash("You have been logged out.")
    return redirect(url_for("login"))

@app.errorhandler(404)
def not_found(e):
    # templates/404.html uses: {{ url_for('static', filename='images/3.jpg') }}
    log.warning("404 Not Found: %s", request.path)
    return render_template("404.html"), 404

if __name__ == "__main__":
    init_db()
    app.run(debug=False)

Created user: admin1
Created user: admin2
Created user: admin3
Created user: admin4
Created user: admin5
 * Serving Flask app '__main__'
 * Debug mode: off


2025-12-04 12:25:53,719 [INFO] WARNING: This is a development server. Do not use it in a production deployment. Use a production WSGI server instead.
 * Running on http://127.0.0.1:5000
2025-12-04 12:25:53,720 [INFO] Press CTRL+C to quit
2025-12-04 12:26:09,218 [INFO] 127.0.0.1 - - [04/Dec/2025 12:26:09] "GET / HTTP/1.1" 200 -
2025-12-04 12:26:09,235 [WARNING] 404 Not Found: /static/images/graph.png
2025-12-04 12:26:09,238 [ERROR] Exception on /static/images/graph.png [GET]
Traceback (most recent call last):
  File "C:\Users\Lily Jayne Baxendale\anaconda3\Lib\site-packages\flask\app.py", line 917, in full_dispatch_request
    rv = self.dispatch_request()
  File "C:\Users\Lily Jayne Baxendale\anaconda3\Lib\site-packages\flask\app.py", line 902, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)  # type: ignore[no-any-return]
           ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~^^^^^^^^^^^^^
  File "C:\Users\Lily Jayne Baxendale\ana